In [2]:
import numpy as np
import xarray as xr

import sys
sys.path.append(r'/home/ch23/ML-BEES_yk/ML-BEES-eval/eval_utilities')
from spatial_temporal_metrics import Metrics
import visualization

In [30]:
# load the ecland data as ground truth (ref)

# select 2021/2022 from Global dataset for the evaluation
# start with ecland_i6aj_o400_2010_2022_6h_euro.zarr
ecland_eval = xr.open_zarr("/data/ecland_i6aj_o400_2010_2022_6h_euro.zarr").sel(time=slice("2021", "2022"))  
# train_ds = xr.open_zarr("/data/ecland_i6aj_o400_2010_2022_6h_euro.zarr")  # Europe subset

# load the ailand data as mod
model_name='euro_mamba_v4_train_2010_2019_val_2020_2020'
ailand_eval = xr.open_zarr("/data/ch23/data_ch23/%s.zarr" % model_name).sel(time=slice("2021", "2022"))  
# new evaluation


In [31]:
# define parameters

mod = ailand_eval
ref = ecland_eval
path='/data/ch23/evalution_results/%s/spatial/' % model_name
path_png='/data/ch23/evalution_results/%s/visualization/' % model_name
figure_path='/data/ch23/evalution_results/%s/visualization/' % model_name

In [32]:
# the problem of chunk in ailand output, need to be rechunked based on the shape of mod

desired_chunks = (4, 10051, 17)  # Adjust based on your desired chunk sizes
mod = mod.chunk({'time': 4, 'x': 10051, 'variable': 17})


In [33]:
# initialize the metrics class -- mc
mc=Metrics(mod,ref,path)

In [6]:
vars = np.intersect1d(mod.variable, ref.variable)

In [34]:
# visualization -- for example: swvl1
########  bias  ###########
bias=mc.bias(relative=False)
for var in bias.variable.values:
    visualization.vis_zarr_map(bias
                               ,var
                               ,figure_path+'bias'
                                ,1,99)

In [35]:
########  nor_bias  ###########
nor_bias=mc.bias(relative=True)
# visualization -- for example: swvl1
for var in nor_bias.variable.values:
    visualization.vis_zarr_map(nor_bias, var, 
                            figure_path+'nor_bias'
                            ,1,99)

In [36]:
########  rmse  ###########
rmse=mc.rmse(relative=False)
# visualization -- for example: swvl1
for var in rmse.variable.values:
    visualization.vis_zarr_map(rmse, var, 
                            figure_path+'rmse'
                            ,1,99)

In [37]:
########  nor_rmse  ###########
nor_rmse=mc.rmse(relative=True)
# visualization -- for example: swvl1
for var in nor_rmse.variable.values:
    visualization.vis_zarr_map(nor_rmse, var, 
                            figure_path+'nor_rmse'
                            ,1,99)

In [38]:
########  acc  ###########
acc1=mc.acc()
# visualization -- for example: swvl1
for var in acc1.variable.values:
    visualization.vis_zarr_map(acc1, var, 
                            figure_path+'acc'
                            ,1,99)

### evaluation between models

In [ ]:
# compare the results between two models
# example in acc
# start with ecland_i6aj_o400_2010_2022_6h_euro.zarr
ecland_eval = xr.open_zarr("/data/ecland_i6aj_o400_2010_2022_6h_euro.zarr").sel(time=slice("2021", "2022"))  
# train_ds = xr.open_zarr("/data/ecland_i6aj_o400_2010_2022_6h_euro.zarr")  # Europe subset

# load the ailand data as mod

ailand_eval1 = xr.open_zarr("/data/ch23/data_ch23/euro_xgb_train_2010_2019_val_2020_2020_diagnostic_v2.zarr").sel(time=slice("2021", "2022"))  

ailand_eval2 = xr.open_zarr("/data/ch23/data_ch23/euro_xgb_train_2010_2019_val_2020_2020_diagnostic_v2.zarr").sel(time=slice("2021", "2022"))  

# define parameters

mod2 = ailand_eval2

path='/data/ch23/evalution_results/euro_xgb_train_2010_2019_val_2020_2020_diagnostic_v2/spatial/'
path_png='/data/ch23/evalution_results/euro_xgb_train_2010_2019_val_2020_2020_diagnostic_v2/visualization/'

mc=Metrics(mod2,ref,path)
acc2=mc.acc()

diff_acc=acc2-acc1

# visualization -- save the diff between models
for var in acc1.variable.values:
    visualization.vis_zarr_map(acc1, var, 
                            figure_path
                            ,1,99)
